In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [3]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7882, 11)

In [4]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

(1534707, 10)

In [5]:
# recuperar todos los pacientes ID desde Contact
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

(1534707, 3)

In [6]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

(1222504, 5)

In [7]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci


In [8]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id,Name_y,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Metlife Holding Metlife Finning,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5951,None,None


In [9]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6224\3766479298.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  Union_account_contact_relation = pd.merge(


,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id_x,Name_y,...,Id_y,IdentificationId__pc_y,Name,poliza__c_y,Identification_Type__c_y,IsPersonAccount,Ocupaci_n__c,Pa_s_del_Paciente__c_y,PersonTitle,PersonDepartment
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Metlife Holding Metlife Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Metlife Holding Metlife Finning,...,0018c00002cVCmMAAW,233986291,Albanez Gonzalez Matias Oliver Franci,None,RUT,True,None,a1U8c000005dAtnEAE,None,None


In [10]:
nuevos = "F:/Empresas SF/carga/sacarid.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

    

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
    # aca se trabaja con el nuevo rut desde rut1 ojooooooo    
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()    
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


,nombres,apellidos,rut1,telefono,email,rut
0,Paulina Roxana,Opazo Carvajal,17202451-3,964050660,popazoc@achs.cl,172024513
1,sonia luzmira,miranda zarzuri,10264052-7,961398226,soniluzmi15@gmail.cl,102640527
2,Claudio Javier,Bugueño Ulloa,16142556-7,987573827,cjbuguenou@achs.cl,161425567
3,sujei naomi,silva day,19361805-7,986641803,sujeinaomi@hotmail.com,193618057
4,Cristóbal Guillermo,Catalán Garrido,20117622-0,982134216,catalancristobal22@gmail.com,201176220


In [11]:
#Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["rut1"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace("-", "")
    # aca se trabaja con el nuevo rut desde rut1 ojooooooo    
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()    
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()

pacientes_y_datos = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
pacientes_y_datos.to_excel("F:/Empresas SF/carga/pacientes_en_account.xlsx", index=False)

In [12]:
#buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7886, 5)

In [13]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1222353, 4)

In [14]:
pacientes_y_datos2 = pd.merge(
    left=pacientes_y_datos,
    right=Pacientesc_SF,
    how="left",
    left_on="Id",
    right_on="AccountId",
)
pacientes_y_datos2.to_excel("F:/Empresas SF/carga/pacientes_en_Contact.xlsx", index=False)

In [15]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"]
)

Final_pacientes_y_empresa = Pacientes_Archivo

Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"].str.title().str.strip()    
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"].str.title().str.strip()  
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower().str.strip()  
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"].str.title().str.strip() 
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"].str.title().str.strip()
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"].str.title().str.strip() 
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"].str.title().str.strip() 
if ("cargo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonTitle"] = Final_pacientes_y_empresa["cargo"].str.title().str.strip() 
if ("division" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonDepartment"] = Final_pacientes_y_empresa["division"].str.title().str.strip() 
if ("telefono" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonMobilePhone"] = Final_pacientes_y_empresa["telefono"]

    

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel("F:/Empresas SF/carga/pacientes_q_no_estan.xlsx", index=False)



In [16]:

Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].astype(str)
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.replace("-", "")
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.upper()
Union_account_contact_relation['IdentificationId__pc_y'] = Union_account_contact_relation['IdentificationId__pc_y'].str.strip()


pacientes_y_datos_relation = pd.merge(
    left=Pacientes_Archivo,
    right=Union_account_contact_relation,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc_y",
)
pacientes_y_datos_relation.head()
pacientes_y_datos_relation.to_excel("F:/Empresas SF/carga/pacientes_en_Acountcontacrelation.xlsx", index=False)

In [17]:
#unir empresas y todas sus caracteristicas


empresas_unidas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
empresas_unidas.shape

(7889, 16)

In [19]:
empresas_unidas.to_excel("F:/Empresas SF/carga/Empresas.xlsx", index=False)

In [20]:
Todos_los_Pacientes_con_empresas = pd.merge(
    left=Paciente_Programas_SF,
    right=empresas_unidas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_y",
)
Todos_los_Pacientes_con_empresas.shape

(1230051, 20)

In [21]:
Todos_los_Pacientes_con_empresas.head()

,Id,Paciente__c,Programas_por_Empresas__c,padre__c,Id_x,Name_x,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c,Id_y,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_y
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Metlife Holding Metlife Serv. De Comerc Y Ases...,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP6008,None,None,a1W8c000007vDPvEAM,0018c00002axQaeAAE,MetLife,Metlife Holding Metlife Serv. De Comerc Y Ases...,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,None,None,Clínica Alemana,a1U8c000005dAtnEAE,MetLife,None,EMP6274,Clínica Alemana,340006119,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,None,None,Clínica Alemana,a1U8c000005dAtnEAE,MetLife,None,EMP6274,Clínica Alemana,340006119,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,None,None,Clínica Alemana,a1U8c000005dAtnEAE,MetLife,None,EMP6274,Clínica Alemana,340006119,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,None,None,Clínica Alemana,a1U8c000005dAtnEAE,MetLife,None,EMP6274,Clínica Alemana,340006119,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353


In [22]:
Todos_los_Pacientes_con_empresas2 = pd.merge(
    left=Pacientes_SF,
    right=Todos_los_Pacientes_con_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
Todos_los_Pacientes_con_empresas2.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6224\165953703.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_y'} in the result is deprecated and will raise a MergeError in a future version.
  Todos_los_Pacientes_con_empresas2 = pd.merge(


(1642466, 30)

In [23]:
Todos_los_Pacientes_con_empresas2.head()

,Id_x,IdentificationId__pc_x,Name,poliza__c_x,Identification_Type__c_x,IsPersonAccount,Ocupaci_n__c,Pa_s_del_Paciente__c_x,PersonTitle,PersonDepartment,...,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c_y,Id_y,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_y
0,0018c00002amLTMAA2,173805004,Ximena Alejandra Muñoz Tudela,None,RUT,True,None,a1U8c000005dAtnEAE,None,None,...,MetLife,None,EMP7790,None,None,a1W8c0000083icPEAQ,0018c00002cXJZeAAO,MetLife,Metlife Vulco Vulco S.A.,A-1075
1,0018c00002amLUGAA2,153338949,Juana Andrea Gallardo Espinoza,None,RUT,True,None,a1U8c000005dAtnEAE,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,Valentina Jahdai Gonzalez Diaz,None,RUT,True,None,a1U8c000005dAtnEAE,None,None,...,Farmacias Cruz Verde,None,EMP6476,Farmacias Cruz Verde,Sin Póliza Asociada,a1W8c0000083j8wEAA,0018c00002cXclVAAS,Farmacias Cruz Verde,Farmacias Cruz Verde,A-2322
3,0018c00002amLUIAA2,169332924,Deayan Nicole Parra Peralta,None,RUT,True,None,a1U8c000005dAtnEAE,None,None,...,Ey,None,EMP4888,None,None,a1W8c0000083laZEAQ,0018c00002eTboVAAS,Ey,EY - EY Audit SpA,A-4489
4,0018c00002amLUJAA2,172957692,Camila Fernanda Velásquez Rojas,None,RUT,True,None,a1U8c000005dAtnEAE,None,None,...,Minera Abra,None,EMP7397,None,None,a1W8c0000083iWhEAI,0018c00002cXJTrAAO,Minera Abra,Minera Abra Holding Minera Abra Minera Abra,A-0721


In [24]:
Todos_los_Pacientes_con_empresas2.shape

(1642466, 30)

In [29]:
#Empresa_Padre__c
Todos_los_Pacientes_con_empresas4 = Todos_los_Pacientes_con_empresas2[Todos_los_Pacientes_con_empresas2['Campa_a__c']== 'MetLife']
#Todos_los_Pacientes_con_empresas4.to_excel("F:/Empresas SF/carga/Metlife.xlsx", index=False)


KeyboardInterrupt: 

In [31]:
Todos_los_Pacientes_con_empresas4.shape
Todos_los_Pacientes_con_empresas4.to_excel("F:/Empresas SF/carga/Metlife.xlsx", index=False)

ValueError: I/O operation on closed file.

Aca es necesario El codigo Id desde Account de la empresa

In [26]:
#AcoountContactRelation
Id_Y id desde account
empresa__c empresa

SyntaxError: invalid syntax (1909426273.py, line 2)

In [27]:
nuevos = "C:/Users/jmartinez/Documents/reporte/19-01/reporte_general.xlsx"
Pacientes_Archivo2 = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

    

Pacientes_Archivo2["rut"] = Pacientes_Archivo2["Rut"].astype(str)
Pacientes_Archivo2["rut"] = Pacientes_Archivo2["rut"].str.replace("-", "")
    # aca se trabaja con el nuevo rut desde rut1 ojooooooo    
Pacientes_Archivo2["rut"] = Pacientes_Archivo2["rut"].str.upper()    
Pacientes_Archivo2["rut"] = Pacientes_Archivo2["rut"].str.strip()
Pacientes_Archivo2.head()

,Ficha Clínica Asistencia Emocional: Ficha Clínica Asistencia Emocional Name,Account,Empresa,Nombre Empresa,Ficha Clínica Asistencia Emocional: Tipo de registro,Ficha Clínica Asistencia Emocional: Fecha de creación,Rut,Programa,rut
0,FCAE-0110,Rosa Luz Acevedo Erasmo,PPN-848000,Farmacias Cruz Verde,E) Ficha Cierre de Servicio,28/11/2022,237550021,Psicología,237550021
1,FCAE-0111,Valeria Elizabe Rojas Parra,PPN-421757,Metlife Holding Metlife Provida,B) Ficha de Seguimiento,28/11/2022,187821444,Psicología,187821444
2,FCAE-0207,Lucas Abrego Peñaranda,NaN,NaN,A) Ficha de Ingreso,1/12/2022,185830438,Psicología,185830438
3,FCAE-0097,Angge Carolay Paico Huaman,PPN-278559,Metlife Holding Metlife Super 10 S.A.,B) Ficha de Seguimiento,26/11/2022,241190153,Psicología,241190153
4,FCAE-0098,Alvaro Bastian Galarce Jerez,PPN-587079,Metlife Holding Metlife Sacyr Asesorias Y Dise...,E) Ficha Cierre de Servicio,26/11/2022,190196356,Psicología,190196356


In [28]:
pacientes_final_reportes = pd.merge(
    left=Pacientes_Archivo2,
    right=Todos_los_Pacientes_con_empresas2,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc_x",
)
pacientes_final_reportes.shape


pacientes_final_reportes.to_excel("C:/Users/jmartinez/Documents/reporte/19-01/salida_reporte_general.xlsx",index=False)